In [1]:
# GITHUB URL - https://github.com/BeeryeIRE/UCDPA_brianconnolly

# In this part of the project, I will demonstrate working with the NHL API to obtain team and player information.
# I will make requests to the API, bring the data into data frames, manipulate the data, set a custom function to look up
# player statistics from the API based on the player's ID, then export an example to a SQL database for querying.

# It should be noted that the NHL API is not publically documented.
# The information regarding NHL API functionality that is available has been established by others via trial and error.
# In particular, I have relied on the documentation carried out by Drew Hynes - see https://github.com/dword4/nhlapi

In [2]:
# Importing the necessary packages for this part of the project

In [3]:
import numpy as np # for data analysis

In [4]:
import pandas as pd # for data analysis

In [5]:
import requests # to make requests to the API

In [6]:
import json # to work with JSON objects

In [7]:
import sqlite3 # to work with SQL databases

In [8]:
import os # for current working directory

In [9]:
# The NHL API does not allow a straight output of player IDs. Instead, the player IDs must be derived from the roster info.
# To get the needed info, I will first need to pull the team info from the relevant endpoint.

teams_url = 'https://statsapi.web.nhl.com/api/v1/teams' # this is the endpoint for team data within the NHL API
teams_resp = requests.get(teams_url) 
teams_json = teams_resp.json() # turn the response object into JSON

In [10]:
# The resulting json dictionary has two keys - copyright and teams. I'm intertesed in "teams"
teams_json.keys()

dict_keys(['copyright', 'teams'])

In [11]:
type(teams_json['teams']) # checking type of teams

list

In [12]:
teams_json['teams'][0] # viewing attributes associated with "team" by looking at the first team - in this case New Jersey

{'id': 1,
 'name': 'New Jersey Devils',
 'link': '/api/v1/teams/1',
 'venue': {'name': 'Prudential Center',
  'link': '/api/v1/venues/null',
  'city': 'Newark',
  'timeZone': {'id': 'America/New_York', 'offset': -5, 'tz': 'EST'}},
 'abbreviation': 'NJD',
 'teamName': 'Devils',
 'locationName': 'New Jersey',
 'firstYearOfPlay': '1982',
 'division': {'id': 18,
  'name': 'Metropolitan',
  'nameShort': 'Metro',
  'link': '/api/v1/divisions/18',
  'abbreviation': 'M'},
 'conference': {'id': 6, 'name': 'Eastern', 'link': '/api/v1/conferences/6'},
 'franchise': {'franchiseId': 23,
  'teamName': 'Devils',
  'link': '/api/v1/franchises/23'},
 'shortName': 'New Jersey',
 'officialSiteUrl': 'http://www.newjerseydevils.com/',
 'franchiseId': 23,
 'active': True}

In [13]:
# the teams dictionary contains other nested dictionaries. I need to flatten the structure to remove nesting.
# I will do this for the first team, before applying a custom function to roll it out to all teams.

NJD_nested = teams_json['teams'][0]

NJD_flat = {key: value for key, value in NJD_nested.items()
            if type(value) is not dict}
NJD_flat

{'id': 1,
 'name': 'New Jersey Devils',
 'link': '/api/v1/teams/1',
 'abbreviation': 'NJD',
 'teamName': 'Devils',
 'locationName': 'New Jersey',
 'firstYearOfPlay': '1982',
 'shortName': 'New Jersey',
 'officialSiteUrl': 'http://www.newjerseydevils.com/',
 'franchiseId': 23,
 'active': True}

In [14]:
# I will now create a function to apply the same flattening to the other teams within the dictionary

def flatten_team(nested):
    flat = {key: value for key, value in nested.items()
           if type(value) is not dict}
    
    flat['venue_name'] = nested['venue']['name']
    flat['venue_city'] = nested['venue']['city']
    flat['franchise_id'] = nested['franchise']['franchiseId']
    flat['division_id'] = nested['division']['id']
    flat['conference_id'] = nested['conference']['id']
    return flat


In [15]:
# using the function to flatten team information and pass it to a data frame.

df_teams = pd.DataFrame([flatten_team(x) for x in teams_json['teams']])

In [16]:
df_teams.head()

,id,name,link,abbreviation,teamName,locationName,firstYearOfPlay,shortName,officialSiteUrl,franchiseId,active,venue_name,venue_city,franchise_id,division_id,conference_id
0,1,New Jersey Devils,/api/v1/teams/1,NJD,Devils,New Jersey,1982,New Jersey,http://www.newjerseydevils.com/,23,True,Prudential Center,Newark,23,18,6
1,2,New York Islanders,/api/v1/teams/2,NYI,Islanders,New York,1972,NY Islanders,http://www.newyorkislanders.com/,22,True,UBS Arena,Elmont,22,18,6
2,3,New York Rangers,/api/v1/teams/3,NYR,Rangers,New York,1926,NY Rangers,http://www.newyorkrangers.com/,10,True,Madison Square Garden,New York,10,18,6
3,4,Philadelphia Flyers,/api/v1/teams/4,PHI,Flyers,Philadelphia,1967,Philadelphia,http://www.philadelphiaflyers.com/,16,True,Wells Fargo Center,Philadelphia,16,18,6
4,5,Pittsburgh Penguins,/api/v1/teams/5,PIT,Penguins,Pittsburgh,1967,Pittsburgh,http://pittsburghpenguins.com/,17,True,PPG Paints Arena,Pittsburgh,17,18,6


In [17]:
df_teams.set_index('id', inplace=True) # we already have an ID field in the dataframe - setting index to this column

In [18]:
# dropping the team site url as it's not necessary.

df_teams.drop('officialSiteUrl', axis=1, inplace=True)
df_teams.drop('link', axis=1, inplace=True)
df_teams.head()

,name,abbreviation,teamName,locationName,firstYearOfPlay,shortName,franchiseId,active,venue_name,venue_city,franchise_id,division_id,conference_id
id,,,,,,,,,,,,,
1,New Jersey Devils,NJD,Devils,New Jersey,1982,New Jersey,23,True,Prudential Center,Newark,23,18,6
2,New York Islanders,NYI,Islanders,New York,1972,NY Islanders,22,True,UBS Arena,Elmont,22,18,6
3,New York Rangers,NYR,Rangers,New York,1926,NY Rangers,10,True,Madison Square Garden,New York,10,18,6
4,Philadelphia Flyers,PHI,Flyers,Philadelphia,1967,Philadelphia,16,True,Wells Fargo Center,Philadelphia,16,18,6
5,Pittsburgh Penguins,PIT,Penguins,Pittsburgh,1967,Pittsburgh,17,True,PPG Paints Arena,Pittsburgh,17,18,6


In [19]:
# Now that I have the team information, I will search for roster information

rosters_url = 'https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster' # this modifier to the teams API endpoint shows the active roster for each team
rosters_resp = requests.get(rosters_url) 
rosters_json = rosters_resp.json() # turn the response object into JSON

In [20]:
# viewing the attributes associated with the rosters dictionary. 
#From this, I am particularly interested in the "person" dictionary, from which I can take each player's unique ID.
# I'll look at the first team in the list, which again is New Jersey.

rosters_json['teams'][0] 

{'id': 1,
 'name': 'New Jersey Devils',
 'link': '/api/v1/teams/1',
 'venue': {'name': 'Prudential Center',
  'link': '/api/v1/venues/null',
  'city': 'Newark',
  'timeZone': {'id': 'America/New_York', 'offset': -5, 'tz': 'EST'}},
 'abbreviation': 'NJD',
 'teamName': 'Devils',
 'locationName': 'New Jersey',
 'firstYearOfPlay': '1982',
 'division': {'id': 18,
  'name': 'Metropolitan',
  'nameShort': 'Metro',
  'link': '/api/v1/divisions/18',
  'abbreviation': 'M'},
 'conference': {'id': 6, 'name': 'Eastern', 'link': '/api/v1/conferences/6'},
 'franchise': {'franchiseId': 23,
  'teamName': 'Devils',
  'link': '/api/v1/franchises/23'},
 'roster': {'roster': [{'person': {'id': 8473541,
     'fullName': 'Jonathan Bernier',
     'link': '/api/v1/people/8473541'},
    'jerseyNumber': '45',
    'position': {'code': 'G',
     'name': 'Goalie',
     'type': 'Goalie',
     'abbreviation': 'G'}},
   {'person': {'id': 8476292,
     'fullName': 'Ondrej Palat',
     'link': '/api/v1/people/8476292'},

In [21]:
NJD_rosterlist = rosters_json['teams'][0]

In [22]:
NJD_roster = NJD_rosterlist['roster']['roster']

In [23]:
# Again, the roster dictionary contains other nested dictionaries. 
# Applying similar function to the one used on the teams dictionary to flatten the structure for use in a dataframe

def flatten_player(nested):
    flat = {}
    flat['person_id'] = nested['person']['id']
    flat['name'] = nested['person']['fullName']
    flat['jersey'] = nested['jerseyNumber']
    flat['position'] = nested['position']['code']
    return flat


In [24]:
# using the function to flatten roster information and pass it to a data frame. This gives us the New Jersey Roster.

NJDroster_df = pd.DataFrame([flatten_player(x) for x in NJD_roster])

In [25]:
NJDroster_df

,person_id,name,jersey,position
0,8473541,Jonathan Bernier,45,G
1,8476292,Ondrej Palat,18,L
2,8478406,Mackenzie Blackwood,29,G
3,8482076,Nico Daws,50,G
4,8474090,Brendan Smith,2,D
5,8475193,Tomas Tatar,90,L
6,8475287,Erik Haula,56,L
7,8476462,Dougie Hamilton,7,D
8,8476923,Damon Severson,28,D
9,8477425,Miles Wood,44,L


In [26]:
# Applying the below function, which takes a team dictionary (with roster field) and returns all the players

def process_roster1(team_dict):
    roster = team_dict['roster']['roster']
    roster = pd.DataFrame([flatten_player(x) for x in roster])
    return roster

In [27]:
# Testing the function on the New Jersey Roster - Team information is missing.

NJD_roster2 = process_roster1(NJD_rosterlist)
NJD_roster2.head()

,person_id,name,jersey,position
0,8473541,Jonathan Bernier,45,G
1,8476292,Ondrej Palat,18,L
2,8478406,Mackenzie Blackwood,29,G
3,8482076,Nico Daws,50,G
4,8474090,Brendan Smith,2,D


In [28]:
# Updating the function to add team information to the dataframe

def process_roster2(team_dict):
    roster = team_dict['roster']['roster']
    df = pd.DataFrame([flatten_player(x) for x in roster])
    df['team_id'] = team_dict['id']
    df['team_name'] = team_dict['name']
    return df

In [29]:
# Testing the updated function again using the New Jersey roster - we can see team_name has been added now.

NJD_roster3 = process_roster2(NJD_rosterlist)
NJD_roster3.head()

,person_id,name,jersey,position,team_id,team_name
0,8473541,Jonathan Bernier,45,G,1,New Jersey Devils
1,8476292,Ondrej Palat,18,L,1,New Jersey Devils
2,8478406,Mackenzie Blackwood,29,G,1,New Jersey Devils
3,8482076,Nico Daws,50,G,1,New Jersey Devils
4,8474090,Brendan Smith,2,D,1,New Jersey Devils


In [30]:
# Now I will apply this function with a concat to create a single dataframe holding roster information for the whole league.

league_rosters = pd.concat([process_roster2(x) for x in rosters_json['teams']], ignore_index=True)

In [31]:
league_rosters.sample(10)

,person_id,name,jersey,position,team_id,team_name
133,8477956,David Pastrnak,88,R,6,Boston Bruins
118,8480172,Jan Rutta,44,D,5,Pittsburgh Penguins
95,8480797,Joel Farabee,86,L,4,Philadelphia Flyers
309,8479542,Brandon Hagel,38,L,14,Tampa Bay Lightning
91,8479772,Zack MacEwen,17,C,4,Philadelphia Flyers
541,8474031,Kevin Shattenkirk,22,D,24,Anaheim Ducks
506,8480802,Ryan McLeod,71,C,22,Edmonton Oilers
213,8482116,Tim Stützle,18,L,9,Ottawa Senators
714,8482149,Cole Perfetti,91,C,52,Winnipeg Jets
43,8480789,Oliver Wahlstrom,26,R,2,New York Islanders


In [32]:
# Checking out the Detroit Redwings Roster via filters to make sure my function has worked.

DRW = league_rosters['team_name'] == 'Detroit Red Wings'
league_rosters_DRW = league_rosters.loc[DRW]
league_rosters_DRW

,person_id,name,jersey,position,team_id,team_name
365,8477952,Robby Fabbri,14,C,17,Detroit Red Wings
366,8479644,Matt Luff,22,R,17,Detroit Red Wings
367,8480821,Filip Zadina,11,R,17,Detroit Red Wings
368,8481725,Elmer Soderblom,85,L,17,Detroit Red Wings
369,8474102,David Perron,57,L,17,Detroit Red Wings
370,8475279,Ben Chiarot,8,D,17,Detroit Red Wings
371,8476874,Olli Maatta,2,D,17,Detroit Red Wings
372,8476897,Oskar Sundqvist,70,C,17,Detroit Red Wings
373,8477330,Dominik Kubalik,81,L,17,Detroit Red Wings
374,8477429,Andrew Copp,18,C,17,Detroit Red Wings


In [33]:
# Now that I have team and player information, I want to lookup some player statistics via the API.
# The NHL API player stats endpoint does not allow you to request information for all players. Instead it requires you to use a single player's ID.
# I will use Steven Stamkos - star player and captain of Tampa Bay - Player ID 8474564

player_id = 8474564

In [34]:
# I will obtain the detailed player statistics on a season by season basis

stats_url = f'https://statsapi.web.nhl.com/api/v1/people/{player_id}/stats?stats=yearByYear' # this is the endpoint for player statistical data within the NHL API, to which I am passing my player_id

In [35]:
stamkos_stats_resp = requests.get(stats_url) # making the request to the API
stamkos_stats_json = stamkos_stats_resp.json() # turn the response object into JSON

In [36]:
# again, there are two dictionaries returned - stats and splits.
# I will look at the available infomration using the first season available.

stamkos_stats = stamkos_stats_json['stats'][0]['splits'][0]
stamkos_stats

{'season': '20052006',
 'stat': {'timeOnIce': '00:00',
  'assists': 92,
  'goals': 105,
  'pim': 87,
  'games': 66,
  'powerPlayTimeOnIce': '00:00',
  'evenTimeOnIce': '00:00',
  'penaltyMinutes': '87',
  'faceOffPct': 0.0,
  'shortHandedTimeOnIce': '00:00',
  'points': 197,
  'shifts': 0},
 'team': {'name': 'Markham', 'link': '/api/v1/teams/null'},
 'league': {'name': 'Minor-ON', 'link': '/api/v1/league/null'},
 'sequenceNumber': 1}

In [37]:
# Applying similar function to the one used on the teams and rosterr dictionaries to flatten the structure for use in a dataframe

def flatten_player_year_stats(stats_dict):
    stats_flat = stats_dict['stat']
    stats_flat['season'] = stats_dict['season']
    stats_flat['team'] = stats_dict['team']['name']
    stats_flat['league'] = stats_dict['league']['name']
    return stats_flat

In [38]:
flatten_player_year_stats(stamkos_stats)

{'timeOnIce': '00:00',
 'assists': 92,
 'goals': 105,
 'pim': 87,
 'games': 66,
 'powerPlayTimeOnIce': '00:00',
 'evenTimeOnIce': '00:00',
 'penaltyMinutes': '87',
 'faceOffPct': 0.0,
 'shortHandedTimeOnIce': '00:00',
 'points': 197,
 'shifts': 0,
 'season': '20052006',
 'team': 'Markham',
 'league': 'Minor-ON'}

In [39]:
# using the function to flatten player information and pass it to a data frame.

stamkos_stats_df = pd.DataFrame([flatten_player_year_stats(x) for x in stamkos_stats_json['stats'][0]['splits']])

In [40]:
stamkos_stats_df

,timeOnIce,assists,goals,pim,games,powerPlayTimeOnIce,evenTimeOnIce,penaltyMinutes,faceOffPct,shortHandedTimeOnIce,...,shortHandedGoals,plusMinus,gameWinningGoals,shots,hits,powerPlayPoints,shotPct,overTimeGoals,shortHandedPoints,blocked
0,00:00,92,105,87,66,00:00,00:00,87,0.00,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,50,42,56,63,NaN,NaN,56,0.00,NaN,...,0.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00:00,8,2,8,6,00:00,00:00,8,0.00,00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,47,58,88,61,NaN,NaN,88,NaN,NaN,...,5.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5,1,4,7,NaN,NaN,4,NaN,NaN,...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1179:51,23,23,39,79,223:57,946:37,39,45.42,09:17,...,0.0,-13.0,1.0,181.0,75.0,17.0,12.71,0.0,0.0,17.0
6,NaN,4,7,6,9,NaN,NaN,6,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1685:14,44,51,38,82,381:18,1193:22,38,47.91,110:34,...,1.0,-2.0,5.0,297.0,68.0,41.0,17.17,1.0,1.0,38.0
8,NaN,1,2,10,5,NaN,NaN,10,0.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1655:56,46,45,74,82,372:53,1256:29,74,46.49,26:34,...,0.0,3.0,8.0,272.0,84.0,36.0,16.54,1.0,0.0,37.0


In [41]:
# Building on this, I will create a custom function to take the player ID and return that player's historical stats from the API

def hist_stats_by_player_year(player_id):
    stats_url = f'https://statsapi.web.nhl.com/api/v1/people/{player_id}/stats?stats=yearByYear'
    stats_resp = requests.get(stats_url)
    stats_json = stats_resp.json()
    return pd.DataFrame([flatten_player_year_stats(x) for x in stats_json['stats'][0]['splits']])

In [42]:
# I will test this on NHL superstar Connor McDavid

McDavid_stats = hist_stats_by_player_year(8478402)
McDavid_stats

,assists,goals,games,points,season,team,league,pim,powerPlayGoals,penaltyMinutes,...,powerPlayPoints,powerPlayTimeOnIce,evenTimeOnIce,faceOffPct,shotPct,overTimeGoals,shortHandedPoints,shortHandedTimeOnIce,blocked,shifts
0,50,27,33,77,20112012,Tor. Marlboros,GTHL,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,65,41,41,106,20112012,Tor. Marlboros,Other,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32,31,17,63,20112012,PEAC,Other,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41,25,63,66,20122013,Erie,OHL,36.0,2.0,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,8,7,14,20122013,Canada,WJ18-A,2.0,5.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,71,28,56,99,20132014,Erie,OHL,20.0,7.0,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,76,44,47,120,20142015,Erie,OHL,48.0,9.0,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,3,7,11,20142015,Canada,WJC-A,0.0,1.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,32,16,45,48,20152016,Edmonton Oilers,National Hockey League,18.0,3.0,18,...,14.0,133:22,681:18,41.23,15.24,1.0,0.0,35:09,10.0,1030.0
9,8,1,10,9,20152016,Canada,WC-A,6.0,0.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# Some values are NaN as not all leagues that McDavid has played in record the same stats

McDavid_stats.isnull().sum()

assists                  0
goals                    0
games                    0
points                   0
season                   0
team                     0
league                   0
pim                      3
powerPlayGoals           6
penaltyMinutes           3
gameWinningGoals         6
shortHandedGoals         6
plusMinus                4
shots                    9
timeOnIce               12
hits                    12
powerPlayPoints         12
powerPlayTimeOnIce      12
evenTimeOnIce           12
faceOffPct              12
shotPct                 12
overTimeGoals           12
shortHandedPoints       12
shortHandedTimeOnIce    12
blocked                 12
shifts                  12
dtype: int64

In [44]:
# I will fill the blank stats with a value of 0 as I don't want to drop whole seasons from my dataset

McDavid_stats.fillna(0) 

,assists,goals,games,points,season,team,league,pim,powerPlayGoals,penaltyMinutes,...,powerPlayPoints,powerPlayTimeOnIce,evenTimeOnIce,faceOffPct,shotPct,overTimeGoals,shortHandedPoints,shortHandedTimeOnIce,blocked,shifts
0,50,27,33,77,20112012,Tor. Marlboros,GTHL,0.0,0.0,0,...,0.0,0,0,0.00,0.00,0.0,0.0,0,0.0,0.0
1,65,41,41,106,20112012,Tor. Marlboros,Other,0.0,0.0,0,...,0.0,0,0,0.00,0.00,0.0,0.0,0,0.0,0.0
2,32,31,17,63,20112012,PEAC,Other,0.0,0.0,0,...,0.0,0,0,0.00,0.00,0.0,0.0,0,0.0,0.0
3,41,25,63,66,20122013,Erie,OHL,36.0,2.0,36,...,0.0,0,0,0.00,0.00,0.0,0.0,0,0.0,0.0
4,6,8,7,14,20122013,Canada,WJ18-A,2.0,5.0,2,...,0.0,0,0,0.00,0.00,0.0,0.0,0,0.0,0.0
5,71,28,56,99,20132014,Erie,OHL,20.0,7.0,20,...,0.0,0,0,0.00,0.00,0.0,0.0,0,0.0,0.0
6,76,44,47,120,20142015,Erie,OHL,48.0,9.0,48,...,0.0,0,0,0.00,0.00,0.0,0.0,0,0.0,0.0
7,8,3,7,11,20142015,Canada,WJC-A,0.0,1.0,0,...,0.0,0,0,0.00,0.00,0.0,0.0,0,0.0,0.0
8,32,16,45,48,20152016,Edmonton Oilers,National Hockey League,18.0,3.0,18,...,14.0,133:22,681:18,41.23,15.24,1.0,0.0,35:09,10.0,1030.0
9,8,1,10,9,20152016,Canada,WC-A,6.0,0.0,6,...,0.0,0,0,0.00,0.00,0.0,0.0,0,0.0,0.0


In [45]:
# Viewing all the columns available

(McDavid_stats.columns.tolist())

['assists',
 'goals',
 'games',
 'points',
 'season',
 'team',
 'league',
 'pim',
 'powerPlayGoals',
 'penaltyMinutes',
 'gameWinningGoals',
 'shortHandedGoals',
 'plusMinus',
 'shots',
 'timeOnIce',
 'hits',
 'powerPlayPoints',
 'powerPlayTimeOnIce',
 'evenTimeOnIce',
 'faceOffPct',
 'shotPct',
 'overTimeGoals',
 'shortHandedPoints',
 'shortHandedTimeOnIce',
 'blocked',
 'shifts']

In [46]:
# Cutting the stats down to the columns that I am most interested in via a new data frame

McDavid_stats_useful = McDavid_stats[['season', 'team','league','assists','goals','points','pim','hits','shots','shotPct','faceOffPct','blocked']]
McDavid_stats_useful

,season,team,league,assists,goals,points,pim,hits,shots,shotPct,faceOffPct,blocked
0,20112012,Tor. Marlboros,GTHL,50,27,77,NaN,NaN,NaN,NaN,NaN,NaN
1,20112012,Tor. Marlboros,Other,65,41,106,NaN,NaN,NaN,NaN,NaN,NaN
2,20112012,PEAC,Other,32,31,63,NaN,NaN,NaN,NaN,NaN,NaN
3,20122013,Erie,OHL,41,25,66,36.0,NaN,NaN,NaN,NaN,NaN
4,20122013,Canada,WJ18-A,6,8,14,2.0,NaN,23.0,NaN,NaN,NaN
5,20132014,Erie,OHL,71,28,99,20.0,NaN,NaN,NaN,NaN,NaN
6,20142015,Erie,OHL,76,44,120,48.0,NaN,NaN,NaN,NaN,NaN
7,20142015,Canada,WJC-A,8,3,11,0.0,NaN,21.0,NaN,NaN,NaN
8,20152016,Edmonton Oilers,National Hockey League,32,16,48,18.0,12.0,105.0,15.24,41.23,10.0
9,20152016,Canada,WC-A,8,1,9,6.0,NaN,15.0,NaN,NaN,NaN


In [47]:
# I will now create a sqlite database to store Connor McDavid's stats from above

conn = sqlite3.connect('hockey.sqlite') # creating the database locally in the current working directory

In [48]:
McDavid_stats_useful.to_sql('McDavid_stats_useful', conn, if_exists='replace', index=False) # reading the dataframe into the databse

In [49]:
# Finally, I will query my new SQL database to find all the seasons where McDavid scored 100 points or more.

pd.read_sql('SELECT season, team, assists, goals, points FROM McDavid_stats_useful WHERE points>99', conn)

,season,team,assists,goals,points
0,20112012,Tor. Marlboros,65,41,106
1,20142015,Erie,76,44,120
2,20162017,Edmonton Oilers,70,30,100
3,20172018,Edmonton Oilers,67,41,108
4,20182019,Edmonton Oilers,75,41,116
5,20202021,Edmonton Oilers,72,33,105
6,20212022,Edmonton Oilers,79,44,123
